# Inference demo
In this notebook, you can extract slide embeddings with out pretrained model TITAN.

In [1]:
import h5py
import torch

from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# load model
model = AutoModel.from_pretrained('MahmoodLab/TITAN', trust_remote_code=True)
model = model.to(device)

2024-12-09 14:52:49.874250: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 14:52:49.896399: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 14:52:50.212756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/user/.conda/envs/gigapath/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxv

In [3]:
# load data
from huggingface_hub import hf_hub_download
demo_h5_path = hf_hub_download(
    "MahmoodLab/TITAN", 
    filename="TCGA_demo_features/TCGA-PC-A5DK-01Z-00-DX1.C2D3BC09-411F-46CF-811B-FDBA7C2A295B.h5",
)
file = h5py.File(demo_h5_path, 'r')
features = torch.from_numpy(file['features'][:])
coords = torch.from_numpy(file['coords'][:])
patch_size_lv0 = file['coords'].attrs['patch_size_level0']
print(patch_size_lv0, features.shape, coords.shape, features.dtype, coords.dtype)

1024 torch.Size([1, 3190, 768]) torch.Size([1, 3190, 2]) torch.float32 torch.int64


In [7]:
# extract slide embedding
with torch.autocast(device_type='cuda', dtype=torch.bfloat16), torch.inference_mode():
    features = features.to(device)
    coords = coords.to(device)
    slide_embedding = model.encode_slide_from_patch_features(features, coords, patch_size_lv0)

In [8]:
slide_embedding.shape

torch.Size([1, 768])

In [9]:
type(patch_size_lv0)

numpy.int64